In [ ]:
!pip install transformers
!pip install transformers
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
!polyglot download morph2.en
!polyglot download morph2.fr
!polyglot download morph2.cs
!polyglot download morph2.hu
!polyglot download morph2.it
!polyglot download morph2.es
!polyglot download morph2.ru
!polyglot download morph2.mn
!polyglot download morph2.la

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 30.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 72.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 18.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.3/126.3 KB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52578 sha256=5d1414923e8a806714d9ce880c38ea2f5b252fc1ce471eff35ace3269cdf5a23
  Stored in directory: /root/.cache/pip/wheels/77/4a/9d/5141018da475375d91dc1af07520b1f2b077579f2f55353afb
Successfully built polyglot
Looking in indexes: https://pypi.org/simple, htt

In [ ]:
import numpy as np
import random
from transformers import BertTokenizer
from transformers import AutoTokenizer

from google.colab import drive
drive.mount('drive')

Mounted at drive


In [ ]:
tokenizer_wp = BertTokenizer.from_pretrained("bert-base-multilingual-uncased")
tokenizer_ulm = AutoTokenizer.from_pretrained("xlm-roberta-large")
tokenizer_bpe = AutoTokenizer.from_pretrained("gpt2")

example_word = 'funniest'

print(tokenizer_ulm.tokenize(example_word))
print(tokenizer_bpe.tokenize(example_word))
print(tokenizer_wp.tokenize(example_word))

['▁fun', 'ni', 'est']
['fun', 'n', 'iest']
['fun', '##nies', '##t']


##Morfessor baseline

In [ ]:
from polyglot.text import Text, Word

def tokenize_morf(w, short_lan):
  w = Word(w, language=short_lan)
  return w.morphemes

def predict_word_morfosser(filename, output_filename, lan, short_lan):
  f_pred = open(output_filename,'w', encoding='utf-8')
  with open(filename, 'r', encoding='utf-8') as f_in:
    for line in f_in:
      parts = line.strip().split('\t')
      ans=[x for x in tokenize_morf(parts[0],short_lan)]
      for i in range(1,len(ans)):
        if ans[i].startswith('@@') is False:
          ans[i]='@@'+ans[i]
      predict = (' ').join(ans)
      f_pred.write(parts[0]+'\t'+predict)
      if len(parts) == 3:
        f_pred.write('\t'+parts[2])
      f_pred.write('\n')
  f_pred.close()

def predict_sentence_morfosser(filename, output_filename, short_lan):
  f_pred = open(output_filename,'w', encoding='utf-8')
  with open(filename, 'r', encoding='utf-8') as f_in:
    for line in f_in:
      parts = line.strip().split('\t')
      tokens = parts[0].split(' ')
      predicts = []
      for token in tokens:
        ans=[x.replace('##','@@') for x in tokenize_morf(token,short_lan)]
        for i in range(1,len(ans)):
          if ans[i].startswith('@@') is False:
            ans[i]='@@'+ans[i]
        predicts.extend(ans)  
      predict = (' ').join(predicts)
      f_pred.write(parts[0]+'\t'+predict)
      if len(parts) == 3:
        f_pred.write('\t'+parts[2])
      f_pred.write('\n')
  f_pred.close()

Morfessor word-level prediction

In [ ]:
path = 'drive/MyDrive/Shared task/evaluation/'
out_file =path+'prediction/Morfessor/'+lan+'.word.test.predictions'
lan = 'lat'
short_code = 'la'
in_file =path+'data/'+lan+'.word.test.gold.tsv'
predict_word_morfosser(in_file, out_file, lan, short_code)
print(lan)

lat


Morfessor sentence-level prediction

In [ ]:
path = 'drive/MyDrive/Shared task/evaluation/'
lan = 'ces'
short_lan = 'cs'
in_file =path+'sentence/tests/'+lan+'.sentence.test.tsv'
out_file =path+'prediction/Morfessor/'+lan+'.sentence.test.predictions'
predict_sentence_morfosser(in_file, out_file, short_lan)

##BPE baseline

In [ ]:
def predict_word_bpe(filename, output, tk):
  f_pred = open(output,'w', encoding='utf-8')
  with open(filename, 'r', encoding='utf-8') as f_in:
    for line in f_in:
      parts = line.strip().split('\t')
      ans=[x.replace('##','@@') for x in tk.tokenize(parts[0])]
      for i in range(1,len(ans)):
        if ans[i].startswith('@@') is False:
          ans[i]='@@'+ans[i]
      predict = (' ').join(ans)
      f_pred.write(parts[0]+'\t'+predict)
      if len(parts) == 3:
        f_pred.write('\t'+parts[2])
      f_pred.write('\n')
  f_pred.close()

BPE word prediction

In [ ]:
path = 'drive/My Drive/Shared task/evaluation/'
lan = 'mon'
in_file =path+'data/'+lan+'.word.dev.tsv'
out_file =path+'prediction/'+lan+'.word.dev.bpe.tsv'
predict_word_bpe(in_file, out_file, tokenizer_wp)

## ULM Baseline

In [ ]:
def predict_word_ulm(filename, output, tk):
  f_pred = open(output,'w', encoding='utf-8')
  with open(filename, 'r', encoding='utf-8') as f_in:
    for line in f_in:
      parts = line.strip().split('\t')
      ans=[x for x in tk.tokenize(parts[0])]
      ans[0]= ans[0][1:]
      for i in range(1,len(ans)):
        if ans[i].startswith('@@') is False:
          ans[i]='@@'+ans[i]
      predict = (' ').join(ans)
      f_pred.write(parts[0]+'\t'+predict)
      if len(parts) == 3:
        f_pred.write('\t'+parts[2])
      f_pred.write('\n')
  f_pred.close()

def predict_sentence_ULM(filename, output_filename, tk):
  f_pred = open(output_filename,'w', encoding='utf-8')
  with open(filename, 'r', encoding='utf-8') as f_in:
    for line in f_in:
      parts = line.strip().split('\t')
      tokens = parts[0].split(' ')
      predicts = []
      for token in tokens:
        ans=[x for x in tk.tokenize(token)]
        if len(ans) == 0:
          ans = [token]
        else:
          ans[0]= ans[0][1:]
        for i in range(1,len(ans)):
          if ans[i].startswith('@@') is False:
            ans[i]='@@'+ans[i]
        predicts.extend(ans)  
      predict = (' ').join(predicts)
      f_pred.write(parts[0]+'\t'+predict)
      if len(parts) == 3:
        f_pred.write('\t'+parts[2])
      f_pred.write('\n')
  f_pred.close()

ULM word prediction

In [ ]:
path = 'drive/MyDrive/Shared task/evaluation/'
lans = ['rus','ita','mon','ces','lat','fra','spa','hun']

for lan in lans:
  in_file =path+'data/'+lan+'.word.test.gold.tsv'
  out_file =path+'prediction/ULM/' +lan+'.word.ulm.pred.tsv'
  print(lan)
  predict_word_ulm(in_file, out_file, tokenizer_ulm)

rus
ita
mon
ces
lat
fra
spa
hun


ULM sentence prediction

In [ ]:
path = 'drive/MyDrive/Shared task/evaluation/'
lan = 'mon'
in_file =path+'sentence/tests/'+lan+'.sentence.test.tsv'
out_file =path+'prediction/ULM/'+lan+'.sentence.test.predictions'
predict_sentence_ULM(in_file, out_file, tokenizer_ulm)